In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
from IPython.display import Image

# Hi!!!

**Hi, I am Sina Tavakoli, today I am trying to get a good example of large scale data. For this we need a fitness function and an algorithm!**

I get my main idea for fitness function from The surface area below the chart.

In [ ]:
Image(filename='../input/photo123/maxresdefault.jpg') 

**I think if we have two distributions, and the area below the diagram of these two distributions in very small intervals is Equal then this two distributions is equal**

Let's go to implement the idea

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA 
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics.pairwise import pairwise_distances_argmin

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df_=pd.read_csv(r'../input/banking-data/Datae2.csv')

df=df_.drop(['id','year','message_code','service','atm_status',
       'atm_manufacturer','atm_location','atm_streetname',
       'atm_street_number','atm_zipcode','weather_lat',"atm_id",
       'weather_lon','weather_city_id','weather_city_name','weather_description'],axis=1)


df['rain_3h']=df['rain_3h'].fillna(0)
df.message_text=df.message_text.fillna("None")

df.message_text = df.message_text.replace(["Suspected malfunction, no cash dispensed","Suspected malfunction, card retained"],"Suspected malfunction")
df.message_text = df.message_text.replace(["Timed-out taking card, card retained and no cash dispensed","Timed-out taking money"],"Timed-out")

# fuzzy encoding!!

I have doubts about whether this is useful and defensible, and I would like you to comment on this.:)

In [ ]:
def fuzzyday(x):
    a=0
    b=0
    c=0

    if x<4:
        a=1
    elif x<9:
        Y=x-4
        a = 1-Y*0.25
    elif x>8:
        a=0
        
    if x<4:
        b=0
    elif x<8:
        T = x-4
        b=T*0.25
    elif x<25:
        b=1
    elif x<29:
        T = x-25
        b=1-T*0.25
        T+=1
        
    if x<25:
        c=0
    elif x<29:
        L=x-25
        c=0.25*L
    elif x>28:
        c=1
    
    
    return round(a*10),round(b*10),round(c*10)

def fuzzyhour(x):
    a=0
    b=0
    c=0

    if x<6:
        a=1
    elif x<11:
        Y=x-5
        a = 1-Y*0.2
    elif x>10:
        a=0
        
    if x<6:
        b=0
    elif x<11:
        T = x-5
        b=T*0.2
    elif x<15:
        b=1
    elif x<21:
        T = x-15
        b=1-T*0.2
        
        
    if x<15:
        c=0
    elif x<20:
        L=x-15
        c=0.20*L
    elif x>20:
        c=1
    
    
    return round(a*10),round(b*10),round(c*10)

# Data pre-processing & PCA
We use pca for continuous distribution that represents the actual distribution! (Our comparison will be easier in this case.)

In [ ]:
df["1day"]=1
df["2day"]=2
df["3day"]=3
df["1day"],df["2day"],df["3day"] = zip(*df.day.apply(lambda x:fuzzyday(x)))
df.drop("day",axis=1, inplace=True)

df["1hour"]=1
df["2hour"]=2
df["3hour"]=3
df["1hour"],df["2hour"],df["3hour"] = zip(*df.hour.apply(lambda x:fuzzyhour(x)))
df.drop("hour",axis=1, inplace=True)


dff = pd.get_dummies(df[["month","weekday","currency","card_type",'weather_main',"message_text"]],drop_first=True)

df = df.drop(["month","weekday","currency","card_type",'weather_main',"message_text"],axis=1)

df = pd.merge(df,dff,left_index=True,right_index=True)

MMS = MinMaxScaler()
MMS.fit(df)
df_norm = MMS.transform(df)
df_norm = pd.DataFrame(df_norm, columns=df.columns)
# df_norm.head()

pca_model = PCA(n_components=0.98)
pca_model.fit(df_norm)
df_pca = pca_model.transform(df_norm)
df_pca = pd.DataFrame(df_pca)


pca_ratio=pca_model.explained_variance_ratio_
pca_ratio

We multiply each column by its **pca_ratio** to give more **importance** to maintaining the distribution of the PC0 than the other columns.

In [ ]:
p=0
df_pca2 = pd.DataFrame()
for j in df_pca.columns:
    df_pca2[j] = df_pca[j]*pca_ratio[p]
    p+=1
    
df_pca2 = df_pca2*10

**And the area below the relative distribution diagram of each feature is stored in the intervals of 0.02**

In [ ]:
distrToTal = []

for j in df_pca.columns:
    distr = []
    arange = []
    pas = 0
    sta = -0.95
    for i in range(2000): 
        lenn =len(df_pca[j][df_pca[j]<sta])
        
        distr.append((lenn / 1250000) - pas)
        pas = np.sum(distr)
        
        arange.append(round(sta,2))
        sta +=0.02
        
        if sta>1.5:
            break
            
    distrToTal.append(distr)
distrToTal[0]

# Fitness

In [ ]:
def fitness(df):
    summy = []
    for j in df.columns:
        j = int(j)
        sc=0
        pas=0
        count=0
        summ=[]
        for i in arange:
            lenn =df[df[j]<i].shape[0]
            pers = (lenn / df.shape[0]) - pas
            pas += pers
            summ.append(np.abs(pers - distrToTal[j][count]))
            count+=1
        summy.append(np.sum(summ) * pca_ratio[j])
        
    return (np.sum(summy)/2)

# Mini batch K-means

In [ ]:
df_pca_mi = df_pca2.copy()
mbkm =MiniBatchKMeans(n_clusters=1700,max_iter=800, batch_size=50000 , verbose=1)

mbkm.fit(df_pca_mi)
df_pca_mi["lable"] = mbkm.predict(df_pca_mi)



list0 = []
for i in range(1900):
    dff = df_pca_mi[df_pca_mi["lable"] == i]
    list1 = np.random.choice(dff.index ,size = int(np.round((len(dff)/10))),replace=False)
    list0.append(list1)
    
    
    
hhh = []
for i in list0:
    for j in i:
        hhh.append(j)
        
        
        
print(len(hhh))


df_sample = df_pca[df_pca.index.isin(hhh)]

print("Matching percentage:",((1-fitness(df_sample))*100) , "%")

# A strange algorithm I made myself:)))))))

In [ ]:
best=10
for i in tqdm(range(20)):
    ind = []

    for i in range(len(arange)-1):
        data = df_pca[(df_pca[0]<arange[i+1]) & (df_pca[0]>arange[i])]
        data = data.iloc[:,1:]
        le = round(len(data)/10)

        if le<500:
            tt = np.random.choice(data.index,le,replace=False)
            ind.append(tt)
        else:
            n_c = round(le/100)
            mbkm =MiniBatchKMeans(n_clusters=n_c,max_iter=800, batch_size=round(le/4) , verbose=0)

            mbkm.fit(data)
            data["lable"] = mbkm.predict(data)



            list3 = []
            for i in range(n_c):
                dff = data[data["lable"] == i]
                list1 = np.random.choice(dff.index ,size = int(np.round((len(dff)/10))),replace=False)
                list3.append(list1)

            hhh = []
            for i in list3:
                for j in i:
                    hhh.append(j)

            ind.append(hhh)
    fina = []
    for i in ind:
        for j in i:
            fina.append(j)
            
    df_sample = df_pca[df_pca.index.isin(fina)] 
    fit = fitness(df_sample)

    if fit<best:
        best = fit
        best_sample=df_sample
        print("Matching percentage:" ,(1-best)*100,"%")

**Apparently, this algorithm that I made gets a better answer
If you enjoyed this notebook, I will be happy to like it**

In [ ]:
print("my best sample is:")
best_sample